<a href="https://colab.research.google.com/github/sdivyakolu/Pytorch_Tutorial/blob/main/PyTorch_DeepLearning_FullModelTraining_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Model Development Traning ( SOLUTIONING THE PROBLEM : Most important step)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as CIFAR10
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import dataloader
from torch.utils.data import random_split
from torch.utils.data import Subset

from torch import optim
import torch.optim as optim

import matplotlib.pyplot as plt

In [5]:
#Define neural network , __init__ and forward functions.

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=nn.Conv2d(3,6, 5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,16*5*5)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x

In [6]:
#Create model class and initiate it
net=Net()
print(net)

#Define loss function and Optimizer
#lr=0.4
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
 #Load and trasnform the data

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(std=(0.5,0.5,0.5),
                                                                    mean=(0.5,0.5,0.5))])

train_trasforms=transforms.Compose([transforms.RandomCrop(32,padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.transforms.ToTensor(),
                                    transforms.Normalize(mean=(0.4914,0.4822,0.4465),
                                                         std=(0.2023,0.1994,0.2010))])

batch_size=32
train_size=400
val_size=100



train_data=CIFAR10.CIFAR10(root='./data',train=True,download=True,transform=train_trasforms)
test_set=CIFAR10.CIFAR10(root='./data',train=False,download=True,transform=transform)
#train_set, validation_set=random_split(train_data,[train_size,val_size])

# Create subsets
train_sub_data = Subset(train_data, range(4000))   # First 400 samples
test_validation_sub_set  = Subset(test_set, range(1000))    # First 100 samples

trainloader=torch.utils.data.DataLoader(train_sub_data,batch_size=batch_size,shuffle=True,num_workers=2)

validation_loader=torch.utils.data.DataLoader(test_validation_sub_set,batch_size=batch_size,shuffle=False,num_workers=2)

#Example:Check the length of splits
print("Traning Set Size:",{len(train_sub_data)})
print("Validation Set Size:",{len(test_validation_sub_set)})


testloader=torch.utils.data.DataLoader(test_validation_sub_set,batch_size=batch_size,shuffle=False,num_workers=2)

Traning Set Size: {4000}
Validation Set Size: {1000}


In [12]:
#Train the network/Model
epochs=10
for epoch in range(epochs):
  net.train() #set the model to training mode
  running_loss=0.0
  for i,data in enumerate(trainloader,0):

    #get the inputs, data is a triple of [inputs,labels]
    inputs,labels=data

    #zero the parameter gradiant
    optimizer.zero_grad()

    #farward + backward = optimize
    outputs=net(inputs)
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    #Print Statstics
    running_loss+=loss.item()
    net.eval()  #set the model to evaluation mode for validation
    validation_loss=0.0
    correct=0
    total=0
    with torch.no_grad():
        for data in testloader:
          images,labels=data
          outputs=net(images)
          loss=criterion(outputs,labels)
          validation_loss +=loss.item()
          _,predicted=torch.max(outputs.data,1)
          total+=labels.size(0)
          correct += ( predicted==labels).sum().item()
  print(f'epoch: {epoch+1}, training_loss: {running_loss/len(trainloader)}, validation_loss: {validation_loss/len(validation_loader)}, accuracy: {100*correct/total}%')
print('Fininsh Traning')

epoch: 1, training_loss: 2.277782241821289, validation_loss: 2.274771213531494, accuracy: 10.4%
epoch: 2, training_loss: 2.2050089025497437, validation_loss: 2.2154005616903305, accuracy: 24.6%
epoch: 3, training_loss: 2.0990346307754515, validation_loss: 2.158205658197403, accuracy: 25.1%
epoch: 4, training_loss: 2.0214395513534544, validation_loss: 2.0931660532951355, accuracy: 27.6%
epoch: 5, training_loss: 1.9820764312744141, validation_loss: 2.084401521831751, accuracy: 27.4%
epoch: 6, training_loss: 1.9542114973068236, validation_loss: 2.0534238442778587, accuracy: 27.6%
epoch: 7, training_loss: 1.919598711013794, validation_loss: 2.0383739806711674, accuracy: 29.7%
epoch: 8, training_loss: 1.8982076368331908, validation_loss: 2.0036757215857506, accuracy: 30.5%
epoch: 9, training_loss: 1.8589166975021363, validation_loss: 2.001039583235979, accuracy: 32.0%
epoch: 10, training_loss: 1.8413868808746339, validation_loss: 1.9769608713686466, accuracy: 32.1%
Fininsh Traning
